# Unit 2: Fisheries Collapse Module Overview

This module will focus on examining a crucial global issue and important scientific debate about the state of global fisheries.  In this module we will seek to reproduce some of the most widely cited examples of species collapse ever, and examine the evidence behind an influential and widely cited paper on global fisheries, [Worm et al 2006](http://doi.org/10.1126/science.1132294).  However, rather than use the limited data available to Boris Worm and colleagues in 2006, we will be drawing from the best and most recent stock asssement data available today to see how those patterns have faired.  

In this module we will also begin to master one of the most important concepts in data science: manipulation of tabular data using relational database concepts. Instead of working with independent data.frames, we will be working with a large relational database which contains many different tables of different sizes and shapes, but that all all related to each other through a series of different ids.  



## The Database

We will use data from the RAM Legacy Stock Assessment Database.  In order to better introduce some important emerging technologies, we will be accessing these data directly from a relatively new platform that is now playing a key role in data sharing in machine learning communities, with the memorable name, HuggingFace.  We will be streaming data from <https://huggingface.co/datasets/cboettig/ram_fisheries/tree/main/v4.65>.  We will have more to say about this approach as we progress.



## Researcher Spotlight: Daniel Pauly

Science is done by real people.  There are many influential and colorful characters in the global fisheries debate.  I want to highlight Professor Pauly not just because he is so famous, but as an early believer in Open Science and Data Science, before we had either of those words.  His contributions in making fisheries data more open were ground breaking for their time.  I'm also indebted to Professor Pauly whom I had the privilege to meet when I was a junior scientist who had only recently released one of my first software packages, aimed at making data from FishBase more accessible. Academic researchers are typically defined by scientific publications, not software, so I was shocked that Pauly already knew of my software package, and that he encouraged me to continue developing software.  Even today that is not common advice, but I believed him, and it's probably a good reason I am where I am today.  Scientific textbooks and courses are often critiqued for failing to recognize the contributions of those from minority backgrounds, but as the texts are written on global change ecology, I think none will omit the works for Professor Pauly.




## Science Introduction

Background abbreviated documentary, features many of the leading authors on both sides https://vimeo.com/44104959

In [2]:
import ibis
from ibis import _
import ibis.selectors as s
import seaborn.objects as so



# Exercise 1: Investigating the North-Atlantic Cod

Now we are ready to dive into our data. First, We seek to replicate the following figure from the Millennium Ecosystem Assessment Project using the RAM data.

![](https://espm-157.github.io/website-r/img/cod.jpg)


In [3]:
con= ibis.duckdb.connect()

tsmetrics_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/tsmetrics.csv?download=true'
area_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/area.csv?download=true'
assessment_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/assessment.csv?download=true'
stock_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/stock.csv?download=true'
time_series_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/timeseries.csv?download=true'



In [18]:


area = con.read_csv(area_url)
tsmetrics= con.read_csv(tsmetrics_url, nullstr = "NA")
assessment = con.read_csv(assessment_url)
stock = con.read_csv(stock_url)
time_series = con.read_csv(time_series_url, nullstr = "NA")



In [19]:
time_series

DatabaseTable: ibis_read_csv_vqez6ojrrvd6dk2oeucvzpzlpi
  assessid  string
  stockid   string
  stocklong string
  tsid      string
  tsyear    int64
  tsvalue   float64

In [5]:
tsmetrics.filter(_.tscategory=='CATCH or LANDINGS').select(_.tslong, _.tsunique).distinct().execute()

,tslong,tsunique
0,Scientific advice for catch limit (pairs with ...,Cadvised-CMB-MT
1,Catch divided by model version only MSY,CdivMSY-mvo-ratio
2,Customary catch,CUSTC-MT
3,Recreational catch,RecC-MT
4,Total allowable catch,TAC-2-MT
...,...,...
63,Total landings. Use only when there is more th...,TL-4-MT
64,Total landings in subarea,TL-A2-MT
65,Total landings in subarea,TL-A4-MT
66,Total landings,TL-E00lbs


In [6]:
time_series.filter(_.stockid== 'COD1ABCDE').select(_.tsid).distinct().execute()

,tsid
0,TL-MT
1,TCbest-MT
2,CdivMEANC-ratio
3,TC-MT


In [11]:
fish= (time_series
    .drop(_.stocklong)
    .rename(tsunique="tsid")
    .join(tsmetrics, "tsunique")
    .join(stock, "stockid")
    .join(assessment, "assessid")
)

fish.head().execute()


,assessid,stockid,tsunique,tsyear,tsvalue,tscategory,tsshort,tslong,tsunitsshort,tsunitslong,...,assesssource,contacts,notes,pdffile,assess,refpoints,assessmethod,assesscomments,xlsfilename,mostrecent
0,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,CdivMEANC-ratio,1960,NA,CATCH or LANDINGS,CdivMEANC,Catch divided by mean catch,ratio,ratio,...,http://adl.brs.gov.au/data/warehouse/9aam/fsrX...,NULL,NULL,00_FishStatus2011_1.0.0,1,1,SS3,NULL,/home/srdbadmin/srdb/spreadsheets/CHING-BGRDRS...,-1
1,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,CdivMEANC-ratio,1961,NA,CATCH or LANDINGS,CdivMEANC,Catch divided by mean catch,ratio,ratio,...,http://adl.brs.gov.au/data/warehouse/9aam/fsrX...,NULL,NULL,00_FishStatus2011_1.0.0,1,1,SS3,NULL,/home/srdbadmin/srdb/spreadsheets/CHING-BGRDRS...,-1
2,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,CdivMEANC-ratio,1962,NA,CATCH or LANDINGS,CdivMEANC,Catch divided by mean catch,ratio,ratio,...,http://adl.brs.gov.au/data/warehouse/9aam/fsrX...,NULL,NULL,00_FishStatus2011_1.0.0,1,1,SS3,NULL,/home/srdbadmin/srdb/spreadsheets/CHING-BGRDRS...,-1
3,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,CdivMEANC-ratio,1963,NA,CATCH or LANDINGS,CdivMEANC,Catch divided by mean catch,ratio,ratio,...,http://adl.brs.gov.au/data/warehouse/9aam/fsrX...,NULL,NULL,00_FishStatus2011_1.0.0,1,1,SS3,NULL,/home/srdbadmin/srdb/spreadsheets/CHING-BGRDRS...,-1
4,ABARES-BGRDRSE-1960-2011-CHING,BGRDRSE,CdivMEANC-ratio,1964,NA,CATCH or LANDINGS,CdivMEANC,Catch divided by mean catch,ratio,ratio,...,http://adl.brs.gov.au/data/warehouse/9aam/fsrX...,NULL,NULL,00_FishStatus2011_1.0.0,1,1,SS3,NULL,/home/srdbadmin/srdb/spreadsheets/CHING-BGRDRS...,-1


In [24]:

cod_catch = (
    fish
    .filter(_.tscategory== "CATCH or LANDINGS")
    .filter(_.tsunique == "TCbest-MT")
    .filter(_.commonname == "Atlantic cod")
)

fish.group_by(_.tslong, _.tsunique).agg(n= _.count()).order_by(_.n.desc()).execute()

cod_catch.columns

['assessid',
 'stockid',
 'tsunique',
 'tsyear',
 'tsvalue',
 'tscategory',
 'tsshort',
 'tslong',
 'tsunitsshort',
 'tsunitslong',
 'tsn',
 'scientificname',
 'commonname',
 'areaid',
 'stocklong',
 'region',
 'primary_country',
 'primary_FAOarea',
 'ISO3_code',
 'GRSF_uuid',
 'GRSF_areaid',
 'inmyersdb',
 'myersstockid',
 'state',
 'assessorid',
 'stockid_right',
 'stocklong_right',
 'recorder',
 'daterecorded',
 'dateloaded',
 'assessyear',
 'assesssource',
 'contacts',
 'notes',
 'pdffile',
 'assess',
 'refpoints',
 'assessmethod',
 'assesscomments',
 'xlsfilename',
 'mostrecent']

In [50]:
cod_tidy = (
    cod_catch.select(_.tsyear, _.tsvalue)
)
cod_tidy.execute()

,tsyear,tsvalue
0,1978,14377
1,1979,12490
2,1980,14567
3,1981,16698
4,1982,26463
...,...,...
7437,2013,4037.8765
7438,2014,6871.9707
7439,2015,6324.331
7440,2016,7195.1255


In [51]:
cod_tidy.group_by('tsyear').tsvalue.mean()

AttributeError: 'GroupedArray' object has no attribute 'mean'

In [36]:
cod_catch.group_by(_.tsyear).agg(n= _.count()).order_by(_.n.desc()).execute()

,tsyear,n
0,2007,152
1,2008,152
2,2009,151
3,2010,149
4,2011,148
...,...,...
168,1898,1
169,1900,1
170,1903,1
171,1906,1


In [ ]:
#cod_catch.filter(_.tsyear == 2007,_.stockid == "CODIIIaW-IV-VIId").execute()

#.group_by(_.stockid).agg(n= _.count()).order_by(_.n.desc()).execute()



,assessid,stockid,tsunique,tsyear,tsvalue,tscategory,tsshort,tslong,tsunitsshort,tsunitslong,...,assesssource,contacts,notes,pdffile,assess,refpoints,assessmethod,assesscomments,xlsfilename,mostrecent
0,WGNSSK-CODIIIaW-IV-VIId-1962-2018-ICESIMP2018,CODIIIaW-IV-VIId,TCbest-MT,2007,52900,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,NA,NA,Imported from ICES data dump covering 2011-2018,NA,1,1,Unknown,ICES Stock cod.27.47d20 in 2018 (Assessment Ke...,ICES.IMPORT.2011-2018.xlsx,0
1,WGNSSK-CODIIIaW-IV-VIId-1962-2019-ICESIMP2021,CODIIIaW-IV-VIId,TCbest-MT,2007,52638,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,http://ices.dk/sites/pub/Publication%20Reports...,NA,Imported from 2021 ICES data dump,NA,1,1,Unknown,ICES Stock cod.27.47d20 in 2019 (Assessment Ke...,ICES.IMPORT.2021.xlsx,0
2,WGNSSK-CODIIIaW-IV-VIId-1962-2022-ICESIMP2024,CODIIIaW-IV-VIId,TCbest-MT,2007,54636.70273,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,https://doi.org/10.17895/ices.advice.21406881.v1,NA,Imported from partial 2024 ICES data dump,NA,1,1,Unknown,ICES Stock cod.27.47d20 in 2022 (Assessment Ke...,ICES.IMPORT.2024.xlsx,999
3,WGNSSK-CODIIIaW-IV-VIId-1963-2020-ICESIMP2021-2,CODIIIaW-IV-VIId,TCbest-MT,2007,52300,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,NA,NULL,Imported from 2nd 2021 ICES bulk import,NULL,1,1,Unknown,ICES Stock cod.27.47d20 in 2020 (Assessment Ke...,ICES.IMPORT.2021-2.xlsx,0
4,WGNSSK-CODIIIaW-IV-VIId-1963-2021-ICESIMP2021-2,CODIIIaW-IV-VIId,TCbest-MT,2007,54600,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,http://www.ices.dk/sites/pub/Publication Repor...,NULL,Imported from 2nd 2021 ICES bulk import,NULL,1,1,Unknown,ICES Stock cod.27.47d20 in 2021 (Assessment Ke...,ICES.IMPORT.2021-2.xlsx,0
5,WGNSSK-CODIIIaW-IV-VIId-1962-2016-ICESIMP2018,CODIIIaW-IV-VIId,TCbest-MT,2007,53200,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,http://www.ices.dk/sites/pub/Publication Repor...,NA,Imported from ICES data dump covering 2011-2018,NA,1,1,Unknown,ICES Stock cod-347d in 2016 (Assessment Key 8052),ICES.IMPORT.2011-2018.xlsx,0
6,WGNSSK-CODIIIaW-IV-VIId-1962-2017-ICESIMP2018,CODIIIaW-IV-VIId,TCbest-MT,2007,53300,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,http://www.ices.dk/sites/pub/Publication Repor...,NA,Imported from ICES data dump covering 2011-2018,NA,1,1,Unknown,ICES Stock cod.27.47d20 in 2017 (Assessment Ke...,ICES.IMPORT.2011-2018.xlsx,0
7,WGNSSK-CODIIIaW-IV-VIId-1962-2011-NEUBAUER,CODIIIaW-IV-VIId,TCbest-MT,2007,65200,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,http://www.ices.dk/reports/ACOM/2011/WGNSSK/Se...,NULL,NULL,NA,1,1,SAM,NA,/home/srdbadmin/srdb/spreadsheets/NEUBAUER-COD...,0
8,WGNSSK-CODIIIaW-IV-VIId-1962-2013-ICESIMP2016,CODIIIaW-IV-VIId,TCbest-MT,2007,45900,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,http://www.ices.dk/sites/pub/Publication%20Rep...,NA,Imported from ICES data dump covering 2011-2016,NA,1,1,Unknown,ICES Stock cod-347d in 2013 (Assessment Key 1786),ICES.IMPORT.2011-2016.xlsx,0
9,WGNSSK-CODIIIaW-IV-VIId-1962-2014-ICESIMP2016,CODIIIaW-IV-VIId,TCbest-MT,2007,45200,CATCH or LANDINGS,TCbest,"General total Catch (TC then TL, MT units only)",MT,Metric tons,...,http://www.ices.dk/sites/pub/Publication Repor...,NA,Imported from ICES data dump covering 2011-2016,NA,1,1,Unknown,ICES Stock cod-347d in 2014 (Assessment Key 4121),ICES.IMPORT.2011-2016.xlsx,0


In [34]:
cod_catch.select(_.tsvalue,_.tsyear,_.primary_country, _.tsunitslong).head(50).execute()

,tsvalue,tsyear,primary_country,tsunitslong
0,38600,1958,Canada,Metric tons
1,66000,1959,Canada,Metric tons
2,50600,1960,Canada,Metric tons
3,66300,1961,Canada,Metric tons
4,65200,1962,Canada,Metric tons
5,68200,1963,Canada,Metric tons
6,63300,1964,Canada,Metric tons
7,71000,1965,Canada,Metric tons
8,68500,1966,Canada,Metric tons
9,54200,1967,Canada,Metric tons


In [28]:
cod_catch.select(_.primary_country,_.region).distinct().execute()

,primary_country,region
0,Denmark,European Union
1,Ireland,European Union
2,France,European Union
3,Greenland,Europe non EU
4,Faroe Islands,Europe non EU
5,Poland,European Union
6,Portugal,Canada East Coast
7,Canada,Canada East Coast
8,Greenland,Canada East Coast
9,United Kingdom,European Union


# Excersise 2: Global Fisheries 

## Stock Collapses

We seek to replicate the temporal trend in stock declines shown in [Worm et al 2006](http://doi.org/10.1126/science.1132294):

![](https://espm-157.github.io/website-r/img/worm2006.jpg)